In [ ]:
########################################################################
#
#-----------------------------------------------------------------------
# This code extracts the contents of new Kickstarter projects--projects 
# whose project end date has not yet passed 
#
# Author - Biruk Abate
#
#-----------------------------------------------------------------------
#
#######################################################################

from urllib.request import urlopen # importing webpage request
from bs4 import BeautifulSoup  # importing beautifulsoup parsing library
import pandas as pd  # importing panda library 
df = pd.DataFrame()   # assigning a user defined (pd) to a panda Dataframe

def soupconverter(url):   #function that accepts and parses url in html format and returns soup
    html = urlopen(url)
    soup = BeautifulSoup(html,'lxml') #lxml format is the html display format
    return soup

def data(url):     #function that accepts url, and returns author's name, backers, fund amount, days_remaining etc..
    soup = soupconverter(url)     #accepts and parses url in html format
    Intro = soup.find("div",{"class":"col-20-24"}).p.get_text() #the "class" under "div" tag is used to idenfify and extract element of Introduction section of webpage 
    intro = Intro.replace('\n','') #To avoid printing out '\n' sign  
    
    head = soup.title.text.replace('\n','') # soup.title.text usually extracts the title of webpage
    
    author = head      #User defined author is assigned to head(title)
    mystring= " "      #User defined string is declared
    mystring = mystring + author # preassigned author is added to the empty mystring
    value = mystring.index('by')  #In the statment contained by the "mystring", mystring.index locates the index of 'by' statement 
    author = mystring[value+3:-14] #Extract the statement of my string 14 digits left from the last word and 3 digits right to index of 'by'
    
    div2 = soup.find("div",{"class":"full-description"}) # locate the class named full discription under div tag 
    about=""  # declare empty about string 
    for data in div2.find_all('p'):# for all paragraphs located in div2 container find paragraphs
        about+=data.text    # add the extracted paragraphs to the empty 'about' string
    
    div1= soup.find("div",{"class":"flex flex-column-lg mb4 mb5-sm"}) # find class named 'flex flex ...' under div tag
    #fund = div1.find("div","mb2-lg")
    fund= div1.span.text  # find text within span tag located in div1 container
    
    pledge = div1.find("div","mb2-lg").find("span",class_="money").text # under div1 container, located span with class 'money' and extract its text
    
    back = div1.find("div","ml5 ml0-lg mb2-lg") # under div1 container, locate div tag with name 'm15 m10...'

    backers = back.div.span.text #After extracting the container for backers, extract text
    
    day = div1.find("div","ml5 ml0-lg") #select day container within div1 container tag
    day_remaining = day.span.text    # Within the container tag, extract the text
    
    url = url[:-14] +'/comments'  #select 14 digits to left last character of URL and append it to the comments
    soup = soupconverter(url)   #Call soupconverter to parse element in HTML format
    
    comments = soup.find_all("li",{"class":"NS_comments__comment"})  # find all the 'li' tags by class name ' NS_comments...'
    output= " "  #Declare an empty string
    for comment in comments: # for all commments in all 'li' tags by class name 'NS_comments...'

        if comment.h3.a.text != author:   # This is to avoid the comments written by the author
            statement = comment.find_all('p')   # select all sentences written by a backer
            for p in statement:    # for all sentences in paragraph, iterate and print out sentences one at a time
                sentence = p.text.replace('\n','') + '~'  # The tilda (~) will allow programmers to extract successive comments 
                output= output + sentence + '  '  # add the comments in the empty output sentence 
    project_status = soup.find("div",class_="grid-row order-2-md hide-lg mb3-md").div.div.text  # select text under div tag by the class name 'grid-row order-2-md'
    #end date
    return intro, head, about,fund, backers,day_remaining,output,author,pledge,project_status #function returns the introduction, title(head), Discription page(about), comments(output), pledged money, project_status

csv_data = pd.read_csv('Newprojects.csv',encoding='latin-1').fillna(0) # read from New project csv
for i in csv_data['URL']:  # select column from csv with 'URL' title
    if i != "URL":   #
        intro, title, about,fund, backers,day_remaining,comment,author,pledge,project_status = data(i) # call data function which returns the contents of webpage
        df =df.append({'title':title,'comment':comment,'Author':author,'About':about,'Short_Discription':intro,  #append the content to a panda file
                       'day_remaining':day_remaining,'Fund':fund,'backers':backers,'pledge':pledge,
                       'Results':project_status
                       ,'URL':li},ignore_index=True)
    
display(df)  # display for programmer the outputs of excel sheets

save="firstData.csv"  # declaring the name of csv file
df.to_csv(save)       # converting the panda 'df' dataframe to csv 